In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# importação de dados

url = 'https://github.com/neylsoncrepalde/projeto_eda_covid/blob/master/covid_19_data.csv?raw=true'

In [ ]:
df = pd.read_csv(url, parse_dates=['ObservationDate', 'Last Update'])
df

In [ ]:
df.dtypes

In [ ]:
# Formatando os Nomes das Colunas

import re

def corrige_colunas(col_name):
     return re.sub(r"[/| ]", "", col_name).lower()

# Teste da Função

corrige_colunas("AdgE/P ou")

In [ ]:
# Corrigindo todas as colunas do df

df.columns =  [corrige_colunas(col) for col in df. columns]

df

In [ ]:
#Brasil - selecionando dados do Brasil

df.loc[df.countryregion == 'Brazil']

In [ ]:
# Pelo menos um caso confirmado

brasil = df.loc[
	(df.countryregion == 'Brazil') &
	(df.confirmed > 0)
]	

brasil

In [ ]:
# Gráfico da evolução de casos confirmados

px.line(brasil,'observationdate', 'confirmed', title='Casos confirmados no Brasil')

In [ ]:
# Novos casos por dia - Programação Funcional

brasil ['novoscasos'] = list(map(

	lambda x: 0 if (x==0) else brasil['confirmed'].iloc[x] - brasil['confirmed'].iloc[x-1], np.arange(brasil.shape[0])

))

px.line(brasil, x='observationdate', y='novoscasos', title='Novos casos por dia')

In [ ]:
# Mortes 

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=brasil.observationdate,y=brasil.deaths,name='Mortes',mode='lines+markers',line={'color':'red'})
)

# Layout

fig.update_layout(title='Mortes por COVID-19 no Brasil')

fig.show()

In [ ]:
# Taxa de Crescimento


def taxa_crescimento(data, variable, data_inicio=None, data_fim=None):
	
	# Se data início for nome, define como a primeira data disponível

	if data_inicio == None:
		data_inicio = data.observationdate.loc[data[variable] > 0].min()

	else:
		data_inicio = pd.to_datetime(data_inicio)

	if data_fim == None:
		data_fim = data.observationdate.iloc[-1]
	
	else:
		data_fim = pd.to_datetime(data_fim)

	# Define os valores do presente e passado
	
	passado = data.loc[data.observationdate == data_inicio, variable].values[0]
	
	presente = data.loc[data.observationdate == data_fim, variable].values[0]

	# Define o número de pontos no tempo que vamos avaliar

	n = (data_fim - data_inicio).days

	# Calculo da taxa

	taxa = (presente/passado)**(1/n) - 1

	return taxa*100	

# Taxa de Crescimento Médio do COVID no Brasil em todo o período

taxa_crescimento(brasil, 'confirmed')

In [ ]:
def taxa_crescimento_diaria(data, variable, data_inicio=None):
  
  if data_inicio == None:
    data_inicio = data.observationdate.loc[data[variable]> 0 ].min()

  else:
    data_inicio = pd.to_datetime(data_inicio) 

  data_fim = data.observationdate.max()

  n = (data_fim - data_inicio).days

  taxas = []
  for i in range(1, n+1):
      taxas.append((data[variable].iloc[i] - data[variable].iloc[i-1]) / data[variable].iloc[i-1] * 100)

  return np.array(taxas)

In [ ]:
tx_dia = taxa_crescimento_diaria(brasil, 'confirmed')

tx_dia

In [ ]:
primeiro_dia = brasil.observationdate.loc[brasil.confirmed > 0].min()

px.line(x=pd.date_range(primeiro_dia, brasil.observationdate.max())[1:], y=tx_dia, title='Taxa de crescimento de casos confirmados no Brasil')

In [ ]:
# Predições

from statsmodels.tsa.seasonal import seasonal_decompose

import matplotlib.pyplot as plt

confirmados = brasil.confirmed
confirmados.index = brasil.observationdate

confirmados

# Decompondo

res = seasonal_decompose(confirmados)

# Plotando

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10,8))

ax1.plot(res.observed)
ax2.plot(res.trend)
ax3.plot(res.seasonal)
ax4.plot(confirmados.index, res.resid)
ax4.axhline(0, linestyle='dashed', c='black')
plt.show()

In [ ]:
# Modelagem Modelo ARIMA

!pip install pmdarima

In [ ]:
from pmdarima.arima import auto_arima

from pmdarima.arima import auto_arima

modelo = auto_arima(confirmados)

# Gráficos

fig = go.Figure(go.Scatter(
	x=confirmados.index, y=confirmados, name='Observados'

))

fig.add_trace(go.Scatter(
	x=confirmados.index, y=modelo.predict_in_sample(), name='Preditos'


))

fig.add_trace(go.Scatter(
	x=pd.date_range('2020-05-20', '2020-06-20'), y=modelo.predict(31), name='Forecast'

))

fig.update_layout(title='Previsão de casos confirmados no Brasil para os próximos 30 dias')

fig.show()

In [ ]:
# Modelagem Modelo de Crescimento

!conda install -c conda-forge fbprophet -y

from fbprophet import Prophet

# Processamentos

train =  confirmados.reset_index() [:-5]
test =  confirmados.reset_index() [-5:]

# Renomeando Colunas

train.rename(columns={'observationdate':'ds', 'confirmed': 'y'}, inplace=True)

test.rename(columns={'observationdate':'ds', 'confirmed': 'y'}, inplace=True)


# Definir Modelo de Crescimento

profeta = Prophet(growth='logistic',changepoints=['2020-03-21', '2020-03-30', '2020-04-25', '2020-05-03', '2020-05-10'])


pop= 211463256
train['cap'] = pop

# Treinar Modelo
profeta.fit(train)

# Previsões para o Futuro

future_dates = profeta_make_future_dataframe(periods=200)

future_dates['cap'] = pop

forecast = profeta.predict (future_dates)


fig = go.Figure()

fig.add_trace(go.Scatter(x=forecast.ds, y=forecast.yhat, name='Predição'))


fig.add_trace(go.Scatter(x=train.ds, y=train.y, name='Observados - Treino'))

fig.update_layout(title='Predições de casos confirmados no Brasil')

fig.show()